In [1]:
import numpy as np
import pandas as pd
import re
import nltk
import datefinder
from datetime import datetime
from nltk.stem import WordNetLemmatizer
from number_parser import parse
from collections import Counter

In [2]:
haunted = pd.read_csv("../Data/haunted_places.csv")
haunted.head()

,city,country,description,location,state,state_abbrev,longitude,latitude,city_longitude,city_latitude
0,Ada,United States,Ada witch - Sometimes you can see a misty blue...,Ada Cemetery,Michigan,MI,-85.504893,42.962106,-85.495480,42.960727
1,Addison,United States,A little girl was killed suddenly while waitin...,North Adams Rd.,Michigan,MI,-84.381843,41.971425,-84.347168,41.986434
2,Adrian,United States,If you take Gorman Rd. west towards Sand Creek...,Ghost Trestle,Michigan,MI,-84.035656,41.904538,-84.037166,41.897547
3,Adrian,United States,"In the 1970's, one room, room 211, in the old ...",Siena Heights University,Michigan,MI,-84.017565,41.905712,-84.037166,41.897547
4,Albion,United States,Kappa Delta Sorority - The Kappa Delta Sororit...,Albion College,Michigan,MI,-84.745177,42.244006,-84.753030,42.243097


*****
# Data Cleaning #
*****

In [3]:
haunted.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10992 entries, 0 to 10991
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   city            10989 non-null  object 
 1   country         10992 non-null  object 
 2   description     10992 non-null  object 
 3   location        10989 non-null  object 
 4   state           10992 non-null  object 
 5   state_abbrev    10992 non-null  object 
 6   longitude       9731 non-null   float64
 7   latitude        9731 non-null   float64
 8   city_longitude  10963 non-null  float64
 9   city_latitude   10963 non-null  float64
dtypes: float64(4), object(6)
memory usage: 858.9+ KB


In [4]:
# drop duplicate rows
haunted = haunted.drop_duplicates()

In [5]:
# fill missing longitude with city_longitude, fill missing latitude with city_latitude
haunted["longitude"] = haunted["longitude"].fillna(haunted["city_longitude"])
haunted["latitude"] = haunted["latitude"].fillna(haunted["city_latitude"])

In [6]:
# fill missing city_longitude with longitude, fill missing city_latitude with latitude
haunted["city_longitude"] = haunted["city_longitude"].fillna(haunted["longitude"])
haunted["city_latitude"] = haunted["city_latitude"].fillna(haunted["latitude"])

In [7]:
# drop missing values
haunted = haunted.dropna()

In [8]:
haunted.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10969 entries, 0 to 10991
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   city            10969 non-null  object 
 1   country         10969 non-null  object 
 2   description     10969 non-null  object 
 3   location        10969 non-null  object 
 4   state           10969 non-null  object 
 5   state_abbrev    10969 non-null  object 
 6   longitude       10969 non-null  float64
 7   latitude        10969 non-null  float64
 8   city_longitude  10969 non-null  float64
 9   city_latitude   10969 non-null  float64
dtypes: float64(4), object(6)
memory usage: 942.6+ KB


*****
# Data Analysis #
*****

## a. audio vidence & b. visual evidence

In [9]:
nltk.download("wordnet")

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\yyz\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [10]:
lemmatizer = WordNetLemmatizer()

def contains_keywords(description, keywords):
    """
    General keyword matching function to check if the description contains specified keywords 
    (e.g., audio_keywords).
    
    Parameters:
    - description (str): The text to be checked.
    - keywords (list): The list of keywords.

    Returns:
    - bool: Returns True if the text contains any keyword, otherwise returns False.
    """

    if isinstance(description, str): 
        # 1. Extract all words
        words = re.findall(r"\b\w+\b", description.lower())
        
        # 2. Generate adjacent word combinations (bigrams)
        bigrams = [" ".join(pair) for pair in zip(words, words[1:])]
        words += bigrams 

        # 3. Lemmatization (Nouns + Verbs)
        lemmatized_words = [
            lemmatizer.lemmatize(word, pos="v") for word in words  
        ]
        lemmatized_words = [
            lemmatizer.lemmatize(word, pos="n") for word in lemmatized_words  
        ]

        # 4. Check if keywords are present
        return any(keyword in lemmatized_words for keyword in keywords)

    return False

### a. Determine whether audio evidence is present (True/False)

In [11]:
audio_keywords = [
    "noise", "sound", "whisper", "scream", "cry", "chant", "voice", "footstep", "laugh",
    "growl", "moan", "knock", "howl", "wail", "whimper", "echo", "bang", "shout", "yell",
    "murmur", "mumble", "hiss", "roar", "groan", "bark", "squeak", "thump", "hum", "rattle",
    "rustle", "buzz", "clap", "clatter", "click", "crackle", "creak", "gasp", "grunt", "gurgle",
    "huff", "pant", "shriek", "snarl", "snicker", "sniff", "snore", "snort", "sob", "squeal",
    "stomp", "thud", "wheeze", "whoop", "hear", "horn", "ring", "tick", "holler", "whistle",
    "laughter", "gibberish"
]

In [12]:
haunted["Audio Evidence"] = haunted["description"].apply(lambda x: contains_keywords(x, audio_keywords))
haunted.head()

,city,country,description,location,state,state_abbrev,longitude,latitude,city_longitude,city_latitude,Audio Evidence
0,Ada,United States,Ada witch - Sometimes you can see a misty blue...,Ada Cemetery,Michigan,MI,-85.504893,42.962106,-85.495480,42.960727,True
1,Addison,United States,A little girl was killed suddenly while waitin...,North Adams Rd.,Michigan,MI,-84.381843,41.971425,-84.347168,41.986434,True
2,Adrian,United States,If you take Gorman Rd. west towards Sand Creek...,Ghost Trestle,Michigan,MI,-84.035656,41.904538,-84.037166,41.897547,False
3,Adrian,United States,"In the 1970's, one room, room 211, in the old ...",Siena Heights University,Michigan,MI,-84.017565,41.905712,-84.037166,41.897547,False
4,Albion,United States,Kappa Delta Sorority - The Kappa Delta Sororit...,Albion College,Michigan,MI,-84.745177,42.244006,-84.753030,42.243097,True


In [13]:
print(haunted["Audio Evidence"].value_counts())

Audio Evidence
False    5813
True     5156
Name: count, dtype: int64


### b. Determine whether image/video/visual evidence is present (True/False)

In [14]:
visual_keywords = [
    # Camera, Photo, Video
    "image", "photo", "picture", "snapshot", "photograph", "frame", "shot", "footage", "recording",
    "record", "video", "clip", "film", "movie", "broadcast", "screen", "tape", "projector", "cctv", 
    "picture", "surveillance", "channel",
    
    # Writing and marking
    "writing", "write", "marking", "mark", "graffiti", "inscription", "etching", "etch", "message", 
    "scribble", "paint"
]

In [15]:
haunted["Image/Video/Visual Evidence"] = haunted["description"].apply(lambda x: contains_keywords(x, visual_keywords))
haunted.head()

,city,country,description,location,state,state_abbrev,longitude,latitude,city_longitude,city_latitude,Audio Evidence,Image/Video/Visual Evidence
0,Ada,United States,Ada witch - Sometimes you can see a misty blue...,Ada Cemetery,Michigan,MI,-85.504893,42.962106,-85.495480,42.960727,True,False
1,Addison,United States,A little girl was killed suddenly while waitin...,North Adams Rd.,Michigan,MI,-84.381843,41.971425,-84.347168,41.986434,True,False
2,Adrian,United States,If you take Gorman Rd. west towards Sand Creek...,Ghost Trestle,Michigan,MI,-84.035656,41.904538,-84.037166,41.897547,False,False
3,Adrian,United States,"In the 1970's, one room, room 211, in the old ...",Siena Heights University,Michigan,MI,-84.017565,41.905712,-84.037166,41.897547,False,True
4,Albion,United States,Kappa Delta Sorority - The Kappa Delta Sororit...,Albion College,Michigan,MI,-84.745177,42.244006,-84.753030,42.243097,True,False


In [16]:
print(haunted["Image/Video/Visual Evidence"].value_counts())

Image/Video/Visual Evidence
False    9828
True     1141
Name: count, dtype: int64


## c. Haunted Places Date

In [17]:
# Defining global date thresholds
MAX_DATE = datetime(2025, 1, 1)

def is_valid_date(date_str):
    """Verify whether the date is within a valid range"""
    try:
        date_obj = datetime.strptime(date_str, "%Y-%m-%d")
        return date_obj <= MAX_DATE
    except:
        return False

def extract_date(description):
    text = str(description).lower()
    
    # Priority 1: Handle full date format
    iso_date_match = re.search(r"\b\d{4}-\d{2}-\d{2}\b", text)
    if iso_date_match:
        date_str = iso_date_match.group()
        if is_valid_date(date_str):
            return date_str

    # Priority 2: Dealing with the ten-year statement
    decade_match = re.search(
        r"\b(?:the\s+)?(19\d{2}|20\d{2})['’]s?\b", 
        text, 
        re.IGNORECASE
    )
    if decade_match:
        year = int(decade_match.group(1))
        if year <= MAX_DATE.year:
            return f"{year}-01-01"

    # Priority 3: Processing month + year combinations
    month_year_match = re.search(
        r"\b(jan|feb|mar|apr|may|jun|jul|aug|sep|oct|nov|dec|january|february|march|april|may|june|july|august|september|october|november|december)[\.,]?\s+(\d{4})\b",
        text,
        re.IGNORECASE
    )
    if month_year_match:
        year = int(month_year_match.group(2))
        if year <= MAX_DATE.year:
            month_str = month_year_match.group(1)[:3].title()
            try:
                month_num = datetime.strptime(month_str, "%b").month
                return f"{year}-{month_num:02d}-01"
            except:
                pass

    # Priority 4: Handle the boot year
    year_match = re.search(
        r"\b(?:in|during|around|year\s+of)\s+(\d{4})\b", 
        text, 
        re.IGNORECASE
    )
    if year_match:
        year = int(year_match.group(1))
        if year <= MAX_DATE.year:
            return f"{year}-01-01"

    # Extracting dates using lenient mode
    try:
        matches = list(datefinder.find_dates(
            text, 
            strict=False,
            base_date=datetime(1800, 1, 1)
        ))
        
        # Double filtering: year range and maximum date
        valid_matches = [
            dt for dt in matches 
            if 1700 <= dt.year <= MAX_DATE.year and dt <= MAX_DATE
        ]
        
        if valid_matches:
            return valid_matches[0].strftime("%Y-%m-%d")
    except:
        pass

    # Final fallback: pure four-digit year
    pure_year = re.search(r"\b(19\d{2}|20\d{2})\b", text)
    if pure_year:
        year = int(pure_year.group(1))
        if year <= MAX_DATE.year:
            return f"{year}-01-01"

    return "2025-01-01"

haunted["Haunted Places Date"] = haunted["description"].apply(extract_date)

In [18]:
haunted.head() #Read the first five lines

,city,country,description,location,state,state_abbrev,longitude,latitude,city_longitude,city_latitude,Audio Evidence,Image/Video/Visual Evidence,Haunted Places Date
0,Ada,United States,Ada witch - Sometimes you can see a misty blue...,Ada Cemetery,Michigan,MI,-85.504893,42.962106,-85.495480,42.960727,True,False,1800-01-03
1,Addison,United States,A little girl was killed suddenly while waitin...,North Adams Rd.,Michigan,MI,-84.381843,41.971425,-84.347168,41.986434,True,False,1800-01-01
2,Adrian,United States,If you take Gorman Rd. west towards Sand Creek...,Ghost Trestle,Michigan,MI,-84.035656,41.904538,-84.037166,41.897547,False,False,2025-01-01
3,Adrian,United States,"In the 1970's, one room, room 211, in the old ...",Siena Heights University,Michigan,MI,-84.017565,41.905712,-84.037166,41.897547,False,True,1970-01-01
4,Albion,United States,Kappa Delta Sorority - The Kappa Delta Sororit...,Albion College,Michigan,MI,-84.745177,42.244006,-84.753030,42.243097,True,False,2025-01-01


In [19]:
print(haunted["Haunted Places Date"].value_counts()) #Look at the distribution of values

Haunted Places Date
2025-01-01    7372
1800-01-01     283
1800-01-02     199
1800-01-03     198
1900-01-01     163
              ... 
1800-05-08       1
1965-01-01       1
1800-05-04       1
1974-04-07       1
1864-11-29       1
Name: count, Length: 361, dtype: int64


## d. Haunted Places Witness Count

### d. Use Number Parser to obtain the number of witnesses

numeric_witness_keywords → These keywords must have a number in front to be considered a valid witness count.
- Example: "3 witnesses" → 3.

count_witness_keywords → These keywords increase the witness count by +1 each time they appear, even without a preceding number.
- Example: "witnesses saw something. More witnesses confirmed it." → "witnesses" appears twice, so the witness count is +2.

In [20]:
# Keywords that must have a number in front to be considered valid
numeric_witness_keywords = ["witness"]

# Keywords that increase the witness count each time they appear
count_witness_keywords = ["witness", "see"]

In [21]:
lemmatizer = WordNetLemmatizer()

def parse_witness_count(description):
    if isinstance(description, str):
        # Parse text and convert numbers in the text into Arabic numerals
        parsed_text = parse(description)  
        
        # Extract all words and numbers
        words = re.findall(r"\b\w+\b", parsed_text.lower())  

        # Lemmatization
        lemmatized_words = [lemmatizer.lemmatize(word, pos="v") for word in words]  
        lemmatized_words = [lemmatizer.lemmatize(word, pos="n") for word in lemmatized_words]  

        # 1. Check for "number + numeric_witness_keywords" pattern
        for i in range(len(lemmatized_words) - 1):
            if lemmatized_words[i].isdigit() and lemmatized_words[i+1] in numeric_witness_keywords:
                num = int(lemmatized_words[i])
                if num < 30:  # Filter out large numbers like years
                    return num  # If a valid "number + keyword" pattern is found, return the number

        # 2. Count occurrences of count_witness_keywords
        witness_count = sum(1 for word in lemmatized_words if word in count_witness_keywords)

        # If the text contains count_witness_keywords but no number, return the count of keyword occurrences
        if witness_count > 0:
            return witness_count  # Increase witness count by +1 for each keyword occurrence
    
    return 0  # If neither pattern is found, return 0

In [22]:
haunted["Haunted Places Witness Count"] = haunted["description"].apply(parse_witness_count)
haunted.head()

,city,country,description,location,state,state_abbrev,longitude,latitude,city_longitude,city_latitude,Audio Evidence,Image/Video/Visual Evidence,Haunted Places Date,Haunted Places Witness Count
0,Ada,United States,Ada witch - Sometimes you can see a misty blue...,Ada Cemetery,Michigan,MI,-85.504893,42.962106,-85.495480,42.960727,True,False,1800-01-03,7
1,Addison,United States,A little girl was killed suddenly while waitin...,North Adams Rd.,Michigan,MI,-84.381843,41.971425,-84.347168,41.986434,True,False,1800-01-01,1
2,Adrian,United States,If you take Gorman Rd. west towards Sand Creek...,Ghost Trestle,Michigan,MI,-84.035656,41.904538,-84.037166,41.897547,False,False,2025-01-01,0
3,Adrian,United States,"In the 1970's, one room, room 211, in the old ...",Siena Heights University,Michigan,MI,-84.017565,41.905712,-84.037166,41.897547,False,True,1970-01-01,2
4,Albion,United States,Kappa Delta Sorority - The Kappa Delta Sororit...,Albion College,Michigan,MI,-84.745177,42.244006,-84.753030,42.243097,True,False,2025-01-01,0


In [23]:
print(haunted["Haunted Places Witness Count"].value_counts())

Haunted Places Witness Count
0    5919
1    3671
2    1011
3     275
4      59
5      24
6       7
7       3
Name: count, dtype: int64


## e. Time of Day

In [24]:
def detect_time_of_day(text):
    text = str(text).lower()
    time_keywords = {
        "Evening": ["evening", "night", "midnight", "dark", 'nighttime', 'nocturnal', 'nightfall', 'sundown', 'darkness', 'late hours', 'overnight', 'witching hour', 'vesper', 'candlelight hours', 'shadowy', ],
        "Morning": ["morning", "dawn", "sunrise", 'daybreak', 'sunup', 'early hours', 'first light', 'crack of dawn', 'pre-dawn', 'break of day', 'aurora', 'cockcrow', 'predawn', 'dewy'],
        "Dusk": ["dusk", "sunset", "twilight", 'gloaming', 'evening twilight', 'dim light', 'shadows lengthen', 'crepuscular', 'fading light']
    }
    
    for time_period, keywords in time_keywords.items():
        if any(keyword in text for keyword in keywords):
            return time_period
    return "Unknown"

haunted["Time of Day"] = haunted["description"].apply(detect_time_of_day)

In [25]:
haunted.head()

,city,country,description,location,state,state_abbrev,longitude,latitude,city_longitude,city_latitude,Audio Evidence,Image/Video/Visual Evidence,Haunted Places Date,Haunted Places Witness Count,Time of Day
0,Ada,United States,Ada witch - Sometimes you can see a misty blue...,Ada Cemetery,Michigan,MI,-85.504893,42.962106,-85.495480,42.960727,True,False,1800-01-03,7,Evening
1,Addison,United States,A little girl was killed suddenly while waitin...,North Adams Rd.,Michigan,MI,-84.381843,41.971425,-84.347168,41.986434,True,False,1800-01-01,1,Unknown
2,Adrian,United States,If you take Gorman Rd. west towards Sand Creek...,Ghost Trestle,Michigan,MI,-84.035656,41.904538,-84.037166,41.897547,False,False,2025-01-01,0,Evening
3,Adrian,United States,"In the 1970's, one room, room 211, in the old ...",Siena Heights University,Michigan,MI,-84.017565,41.905712,-84.037166,41.897547,False,True,1970-01-01,2,Evening
4,Albion,United States,Kappa Delta Sorority - The Kappa Delta Sororit...,Albion College,Michigan,MI,-84.745177,42.244006,-84.753030,42.243097,True,False,2025-01-01,0,Unknown


In [26]:
print(haunted["Time of Day"].value_counts())

Time of Day
Unknown    6954
Evening    3819
Morning     169
Dusk         27
Name: count, dtype: int64


## f. Apparition type

### f. Determine the Type of Apparition Present (Ghost, Orb, UFO, etc.)

In [27]:
category_mapping = {
    "ghosts": "Several Ghosts",
    "spirits": "Several Ghosts",
    "figures": "Several Ghosts",
    "hauntings": "Several Ghosts",
    "shadows of teenager": "Several Ghosts",
    
    "ghost": "Ghost",
    "spirit": "Ghost",
    "phantom": "Ghost",
    "specter": "Ghost",
    "apparition": "Ghost",
    "poltergeist": "Ghost",
    "ghostly": "Ghost",
    
    "orb": "Orb",
    "white ball": "Orb",
    "fireball": "Orb",

    "ufo": "UFO",
    "Unidentified Flying Object": "UFO",
    
    "uap": "UAP",
    "Unidentified Aerial Phenomena": "UAP",

    "male": "Male",
    "man": "Male",
    "he": "Male",
    
    "female": "Female",
    "woman": "Female",
    "lady": "Female",
    "she": "Female",

    "child": "Child",
    "kid": "Child",
    "boy": "Child",
    "girl": "Child",

    "teen": "Teenager",
    "teenager": "Teenager",
    "adolescent": "Teenager",

    "human": "Human"
}

In [28]:
lemmatizer = WordNetLemmatizer()

def detect_apparition_type(description):
    if isinstance(description, str):  
        words = re.findall(r"\b\w+\b", description.lower()) 
        bigrams = [" ".join(pair) for pair in zip(words, words[1:])] 
        words += bigrams  

        # 1. Directly match original words (without lemmatization)
        for keyword, category in category_mapping.items():
            if keyword in words:
                return category 
        
        # 2. If no match is found, perform lemmatization
        lemmatized_words = [lemmatizer.lemmatize(word, pos="v") for word in words]
        lemmatized_words = [lemmatizer.lemmatize(word, pos="n") for word in lemmatized_words]

        for keyword, category in category_mapping.items():
            if keyword in lemmatized_words:
                return category 

    # 3.If still no match, return "Unknown"
    return "Unknown"

In [29]:
haunted["Apparition Type"] = haunted["description"].apply(detect_apparition_type)
haunted.head()

,city,country,description,location,state,state_abbrev,longitude,latitude,city_longitude,city_latitude,Audio Evidence,Image/Video/Visual Evidence,Haunted Places Date,Haunted Places Witness Count,Time of Day,Apparition Type
0,Ada,United States,Ada witch - Sometimes you can see a misty blue...,Ada Cemetery,Michigan,MI,-85.504893,42.962106,-85.495480,42.960727,True,False,1800-01-03,7,Evening,Several Ghosts
1,Addison,United States,A little girl was killed suddenly while waitin...,North Adams Rd.,Michigan,MI,-84.381843,41.971425,-84.347168,41.986434,True,False,1800-01-01,1,Unknown,Female
2,Adrian,United States,If you take Gorman Rd. west towards Sand Creek...,Ghost Trestle,Michigan,MI,-84.035656,41.904538,-84.037166,41.897547,False,False,2025-01-01,0,Evening,Several Ghosts
3,Adrian,United States,"In the 1970's, one room, room 211, in the old ...",Siena Heights University,Michigan,MI,-84.017565,41.905712,-84.037166,41.897547,False,True,1970-01-01,2,Evening,Unknown
4,Albion,United States,Kappa Delta Sorority - The Kappa Delta Sororit...,Albion College,Michigan,MI,-84.745177,42.244006,-84.753030,42.243097,True,False,2025-01-01,0,Unknown,Female


In [30]:
print(haunted["Apparition Type"].value_counts())

Apparition Type
Unknown           2696
Ghost             2677
Male              1700
Several Ghosts    1443
Female            1297
Child              950
Orb                148
Human               33
Teenager            22
UFO                  3
Name: count, dtype: int64


## g. Event type

In [31]:
def classify_event(text):
    text = str(text).lower()
    event_categories = {
        "Murder": ["murder", "killed", "stabbed", "homicide", 'slay', 'assassination', 'ritual killing'],
        "Death": ["died", "death", "suicide", "passed away", 'passed on', 'deceased', 'rest in peace', 'met their end'],
        "Supernatural": ["ghost", "spirit", "haunted", "apparition", "paranormal", 'phantom', 'poltergeist' ]
    }
    
    for event_type, keywords in event_categories.items():
        if any(keyword in text for keyword in keywords):
            return event_type
    return "Unknown"

haunted["Event type"] = haunted["description"].apply(classify_event)

In [32]:
haunted.head()

,city,country,description,location,state,state_abbrev,longitude,latitude,city_longitude,city_latitude,Audio Evidence,Image/Video/Visual Evidence,Haunted Places Date,Haunted Places Witness Count,Time of Day,Apparition Type,Event type
0,Ada,United States,Ada witch - Sometimes you can see a misty blue...,Ada Cemetery,Michigan,MI,-85.504893,42.962106,-85.495480,42.960727,True,False,1800-01-03,7,Evening,Several Ghosts,Murder
1,Addison,United States,A little girl was killed suddenly while waitin...,North Adams Rd.,Michigan,MI,-84.381843,41.971425,-84.347168,41.986434,True,False,1800-01-01,1,Unknown,Female,Murder
2,Adrian,United States,If you take Gorman Rd. west towards Sand Creek...,Ghost Trestle,Michigan,MI,-84.035656,41.904538,-84.037166,41.897547,False,False,2025-01-01,0,Evening,Several Ghosts,Murder
3,Adrian,United States,"In the 1970's, one room, room 211, in the old ...",Siena Heights University,Michigan,MI,-84.017565,41.905712,-84.037166,41.897547,False,True,1970-01-01,2,Evening,Unknown,Unknown
4,Albion,United States,Kappa Delta Sorority - The Kappa Delta Sororit...,Albion College,Michigan,MI,-84.745177,42.244006,-84.753030,42.243097,True,False,2025-01-01,0,Unknown,Female,Death


In [33]:
print(haunted["Event type"].value_counts())

Event type
Unknown         4278
Supernatural    3319
Death           1840
Murder          1532
Name: count, dtype: int64


In [34]:
haunted.to_csv("../Data/haunted_place_new.csv", index=False, encoding="utf-8-sig")